

### Importing libraries
Python imports:

In [1]:
#!pip install spacy
import spacy
import string
import re
import itertools
import csv
import shutil
import matplotlib.pyplot as plt
import zipfile
import lzma
import json
import requests
import os
import xml.etree.cElementTree as ET
import pandas as pd
import numpy as np
import seaborn as sns
import random
from nltk.corpus import stopwords 

#!pip install -U spacy unidecode

from unidecode import unidecode
from spacy.matcher import Matcher 
from spacy import displacy 
from IPython.display import Image, display
import nltk
nltk.download('wordnet')
nltk.download('words')
nltk.download('punkt')

from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.util import skipgrams
from nltk.corpus import wordnet as wn
from spacy.lang.en import English

from tqdm import tqdm
from ipywidgets import FloatProgress
%matplotlib inline


#!pip install pyquery
#!pip install -U gensim

import gensim
#import pyLDAvis.gensim
import pickle
from gensim import corpora
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.models import CoherenceModel

[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading words: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


In [2]:
#!pip install pyLDAvis
import pyLDAvis.gensim_models  # don't skip this

In [3]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Downloading data
The Dataset portion we are going to use for the project is the Illinois Bulk Dataset, composed by
183146 CDs. Every CD can report one or more ‘opinions’, that are parts of the document where
one or more judges express their argumentations and indeed, ‘opinions' on the real case.

In [ ]:
download_url = "https://case.law/download/bulk_exports/latest/by_jurisdiction/case_text_open/ill/ill_text.zip"
output_path = "C:/Users/camilla.gotta/Downloads/Illinois-20200302-text.zip"
print("Downloading to %s ..." % output_path)
with open(output_path, 'wb') as out_file:
    shutil.copyfileobj(requests.get(download_url, stream=True).raw, out_file)
print("Done.")

# Exploring data analysis 

The used dataset in this project is Harvard Caselaw Access Project. The Dataset portion we are going to use for the project is the Illinois Bulk Dataset, composed by 183146 CDs (https://case.law/bulk/download/). 

All CDs report the followings metadata: case body, data, attorneys, head_matter, judges, author, opinions (with the dissenting opinions where present), citations, court identification details, date,case ID, jurisdiction, names, reporter, volumes, and others.

In [ ]:
#Our bulk export is stored as a zipped folder in BagIt format, which in turn contains an zipped file in jsonlines format. 
#We can peel off all of those layers and get a list of cases with built-in Python libraries

# a list to hold the cases we're sampling

cases = []

# decompress the file line by line     
with zipfile.ZipFile('C:/Users/hadjiandrea/PycharmProjects/InformationRetrieval/ill_text_20200604.zip', 'r') as zip_archive:
    xz_path = next(path for path in zip_archive.namelist() if path.endswith('/data.jsonl.xz'))
    with zip_archive.open(xz_path) as xz_archive, lzma.open(xz_archive) as jsonlines:
        for line in tqdm(jsonlines):
            # decode the file into a convenient format
            record = json.loads(str(line, 'utf-8'))
            # if the decision date on the case matches one we're interested in, add to our list
            if int(record['decision_date'][:4]):
                cases.append(record)
print("Number of Cases: {}".format(len(cases)))

In [ ]:
#Our bulk export is stored as a zipped folder in BagIt format, which in turn contains an zipped file in jsonlines format. 
#We can peel off all of those layers and get a list of cases with built-in Python libraries

# a list to hold the cases we're sampling

cases = []

# decompress the file line by line     
with zipfile.ZipFile('C:/Users/camilla.gotta/Downloads/Illinois-20200302-text.zip', 'r') as zip_archive:
    xz_path = next(path for path in zip_archive.namelist() if path.endswith('/data.jsonl.xz'))
    with zip_archive.open(xz_path) as xz_archive, lzma.open(xz_archive) as jsonlines:
        for line in tqdm(jsonlines):
            # decode the file into a convenient format
            record = json.loads(str(line, 'utf-8'))
            # if the decision date on the case matches one we're interested in, add to our list
            if int(record['decision_date'][:4]):
                cases.append(record)
print("Number of Cases: {}".format(len(cases)))

Will be used:
- `id`(assigned by CAP database): A unique case identifier that we can use to link opinions belonging to the same case
- `name`: The case's name
- `court`: The court in which the case was heard and decided
- `citations`: The official citation to the case
- `author`: The author of the opinion, if known
- `type`: The type of the opinion (ex. 'majority,''dissent,''concurrence')
- `text`: The full text of the opinion

In [ ]:
# use a list comprehension to pull out the metadata attributes specified above
opinion_data = []
for case in cases:
    for opinion in case["casebody"]["data"]["opinions"]:
        opinion_data.append({'id': case['id'],
                             'year': case['decision_date'],
                             'name': case['name'],
                             'citation': case['citations'][0]['cite'],
                             'court': case['court']['name'],
                             'author': opinion['author'],
                             'type': opinion['type'],
                             'text': opinion['text'],})

In [ ]:
opinion_data[0]

In [ ]:
opinion_data = pd.DataFrame(opinion_data)

In [ ]:
opinion_data = opinion_data.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
opinion_data.head()

In [ ]:
with open("opinion_data.pkl", "wb") as f:
    pickle.dump(opinion_data, f)

In [4]:
#Reload save pkl file
with open('opinion_data.pkl', 'rb') as f:
    opinion_data = pickle.load(f)

In [5]:
opinion_data

,id,year,name,citation,court,author,type,text
0,1156845,1996-03-27,"elsie ghere, widow of jim ghere, deceased, app...",278 ill. app. 3d 840,illinois appellate court,justice colwell,majority,justice colwell\ndelivered the opinion of the ...
1,1156786,1996-02-16,"robert l. o'neil et al., plaintiffs-appellees ...",278 ill. app. 3d 327,illinois appellate court,presiding justice zwick,majority,presiding justice zwick\ndelivered the opinion...
2,1156800,1996-02-26,"in re l.n., a minor (the people of the state o...",278 ill. app. 3d 46,illinois appellate court,justice slater,majority,justice slater\ndelivered the opinion of the c...
3,1156841,1996-02-13,"the people of the state of illinois, plaintiff...",278 ill. app. 3d 218,illinois appellate court,justice scariano,majority,justice scariano\ndelivered the opinion of the...
4,1156843,1996-03-28,"robert alan shramuk, plaintiff-appellant, v. c...",278 ill. app. 3d 745,illinois appellate court,justice geiger,majority,justice geiger\ndelivered the opinion of the c...
...,...,...,...,...,...,...,...,...
197781,2866138,1967-05-18,"raymond pedrick et al., appellants, vs. the pe...",37 ill. 2d 494,illinois supreme court,mr. justice underwood,majority,mr. justice underwood\ndelivered the opinion o...
197782,5550081,1988-02-11,"the people of the state of illinois, appellee,...",122 ill. 2d 176,illinois supreme court,justice ryan,majority,justice ryan\ndelivered the opinion of the cou...
197783,5550081,1988-02-11,"the people of the state of illinois, appellee,...",122 ill. 2d 176,illinois supreme court,"justice simon,",concurring-in-part-and-dissenting-in-part,"justice simon,\nconcurring in part and dissent..."
197784,5426498,1976-03-29,"first capitol mortgage corporation, appellee, ...",63 ill. 2d 128,illinois supreme court,mr. justice ryan,majority,mr. justice ryan\ndelivered the opinion of the...


In [21]:
#drange = pd.date_range('1850-01-01', '2020-01-01', freq='5Y')
court_year = opinion_data['year'].value_counts().sort_index() #.plot.bar()
#plt.savefig('court_year.png')
court_year

1771-10-12     1
1819-12        5
1820-07        6
1820-12        6
1822-12       16
              ..
2019-07-01     2
2019-07-02     1
2019-07-03     1
2019-07-11     3
2019-07-29     1
Name: year, Length: 17958, dtype: int64

In [ ]:
court_lengths = opinion_data['court'].value_counts().sort_index() #.plot.bar()
#plt.savefig('court_lengths.png')
court_lengths

In [ ]:
court_type = opinion_data['type'].value_counts().sort_index() #.plot.bar()
#plt.savefig('court_type.png')
court_type

### Extracting and Cleaning Data

The exact strategy you employ to clean data can be very dependent on the type of analysis you want to do in this one we will cover a few common strategies:

removing non alphanumeric characters
removing common stop words
removing very short or long words
removing purely numerical data
removing words that are not in a particular corpus (such as the oxford english dictionary)

Again, these are common techniques, but depending on your study and data, they may not be necessary or even advisable. The strategy you take will emerge from your data and the type of analysis you plan to do.

### Removing non alphanumeric characters
This loop uses the gemsin library to strip non alphanumeric characters. It also collapses multiple spaces into a single space.

In [ ]:
opinion_texts = []
for i in range(len(cases)):
    if cases[i]['casebody']['data']['opinions']:
        text = cases[i]['casebody']['data']['opinions'][0]['text'].lower()
        text = strip_non_alphanum(text)
        opinion_texts.append(' '.join(text.split()))

In [ ]:
opinion_texts[0]

       
Here's the output from the first record. Take a look at this as compared to the text from the raw JSON file. Non alphanumeric characters are gone, but you can still read the text and understand what it means. However, you may already be able to see how cleaning data can result in loss of precision.

As an exercise, take a look at the original document and consider if there are any terms you'd be interested in preserving that may be lost through the data transformation here.



### Remove stopwords
The above text still contains a large number of "stop words", common words that may not too ubiquitous to be useful (emphasis on the may). Words like "a", "they", "and" increase the word count and require extra processing time and computing resources, and may not be helpful in your analysis.

The loop below uses the gensim utility to remove common stopwords

In [ ]:
for i in range(len(opinion_texts)):
    opinion_texts[i] = remove_stopwords(opinion_texts[i])
#opinion_texts[0]

### Remove very long or short words, Remove purely numerical data
The text still contains a large number of short or purely numeric characters. These may or may not contain valuable information - for now, we'll remove them.

In [ ]:
for i in range(len(opinion_texts)):
    opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if not (len(s) < 2 or s.isdigit() == True)])
#opinion_texts[0]

### Removing words that are not in a particular corpus (such as the oxford english dictionary)

You may have noticed that there are a number of words such as "jj" above that are nor part of the standard english language. These may be the result of data cleaning (such as artifacts of markup language). They may also be the result of legal language or other important data. As always, cleaning data is a decision you make that may or may not be necessary or desirable.


In [ ]:
words = set(nltk.corpus.words.words())
for i in range(len(opinion_texts)):
    opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s in words])
#opinion_texts[0]

In [ ]:
authors = set(nltk.word_tokenize(str(opinion_data['author'].tolist())))
year = set(opinion_data['year'].tolist())
name = set(nltk.word_tokenize(str(opinion_data['name'].tolist())))
type_judge = set(opinion_data['type'].tolist())
court = set(nltk.word_tokenize(str(opinion_data['court'].tolist())))
citation = set(nltk.word_tokenize(str(opinion_data['citation'].tolist())))

In [ ]:
import itertools
#final = list(itertools.chain(authors, year, name, type_judge, court, citation))

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in authors])

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in year])

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in name])

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in type_judge])

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in court])

In [ ]:
for i in range(len(opinion_texts)):
               opinion_texts[i] = ' '.join([s for s in opinion_texts[i].split() if s not in citation])

In [ ]:
#Save file to pkl
with open("opinion_texts.pkl", "wb") as f:
    pickle.dump(opinion_texts, f)

###  Tokenize the sentences
We've cleaned the data, but it is still stored in large blocks of text. The next step, tokenization, will convert each line into a list of individual words.

In [ ]:
#Reload save pkl file
with open('opinion_texts.pkl', 'rb') as f:
    opinion_texts = pickle.load(f)

In [ ]:
opinion_texts = pd.read_pickle("opinion_texts.pkl")

In [ ]:
opinion_texts = pd.DataFrame(opinion_texts)
opinion_texts

In [ ]:
from nltk import word_tokenize

tokenized_sentences = [] 
for text in opinion_texts:
    tokenized_sentences.append(word_tokenize(text))

In [ ]:
tokenized_sentences[0][:50]

In [ ]:
with open("tokenized_sentences.pkl", "wb") as f:
    pickle.dump(tokenized_sentences, f)

### Lemmatization

In [ ]:
with open('tokenized_sentences.pkl', 'rb') as f:
    tokenized_sentences = pickle.load(f)

In [ ]:
import sys
!{sys.executable} -m spacy download en
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [ ]:
# Do lemmatization keeping only noun, adj
data_lemmatized_2 = lemmatization(tokenized_sentences, allowed_postags=['NOUN', 'ADJ'])
#print(data_lemmatized[:1])

In [ ]:
with open("data_lemmatized_2.pkl", "wb") as f:
    pickle.dump(data_lemmatized_2, f)